<a href="https://colab.research.google.com/github/flash1405/GenderBiasLanguageModelling/blob/main/GenderBias_LanguageModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install nltk
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import csv
f = open('/content/drive/MyDrive/Lumiere Datasets/Eval.csv', 'w')
writer = csv.writer(f)
male_file = open("/content/drive/MyDrive/Lumiere Datasets/MaleOccupations.txt")
male_string = male_file.read().replace("\n", " ")
male_list = nltk.tokenize.word_tokenize(male_string)
for i in male_list:
  temp = ['The ' + i + ' said that','he']
  writer.writerow(temp)
female_file = open("/content/drive/MyDrive/Lumiere Datasets/FemaleOccupations.txt")
female_string = female_file.read().replace("\n", " ")
female_list = nltk.tokenize.word_tokenize(female_string)
for i in female_list:
  temp = ['The ' + i + ' said that','she']
  writer.writerow(temp)


In [ ]:
main_file = open("/content/drive/MyDrive/Lumiere Datasets/Harry Potter/J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt")
main_string = main_file.read().replace("\n", " ")
#main_string="Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense."
main_list = nltk.tokenize.sent_tokenize(main_string)
#main_list = ["My name is Aryan."]
input_data = []
output_data = []
def findnth(string, substring, n):
    parts = string.split(substring, n + 1)
    if len(parts) <= n + 1:
        return -1
    return len(string) - len(parts[-1]) - len(substring)
for i in main_list:
  split_space_index=findnth(i," ",int(i.count(" ")/2))
  input_data.append(i[0:split_space_index])
  output_data.append(i[split_space_index+1:-1])
# print(input_data)
# print(output_data)
# print(main_list)

In [ ]:
a = "He she her his"
words = a.split()
for i in range(len(words)):
  if(words[i]=="she"):
    words[i] = "he"
  elif(words[i]=="She"):
    words[i] = "He"
  elif(words[i]=="he"):
    words[i] = "she"
  elif(words[i]=="He"):
    words[i] = "She"
  elif(words[i]=="him"):
    words[i] = "her" 
  elif(words[i]=="Him"):
    words[i] = "Her"
  elif(words[i]=="her"):
    words[i] = "him"
  elif(words[i]=="Her"):
    words[i] = "Him"
  elif(words[i]=="his"):
    words[i] = "hers"
  elif(words[i]=="His"):
    words[i] = "Hers"
  elif(words[i]=="hers"):
    words[i] = "his"
  elif(words[i]=="Hers"):
    words[i] = "His"
print(" ".join(words))

She he him hers


In [ ]:
!pip install transformers
!pip install torch

In [ ]:
from transformers import pipeline
generator = pipeline("text-generation",model="distilgpt2")
generator("Harry ",max_lenght=1,num_return_sequences=2,)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Harry ***********************\nThe first half was absolutely perfect and I could only think of my thoughts because it\u200d“ was so so sweet.\nI heard that he still wanted to be a boy and that the only way to fight was'},
 {'generated_text': 'Harry ---------------------------\n\n› The urchin has been left behind\n› The urchin is still holding water. › The urchin is still holding water while its egg is still standing.› The urch'}]

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
classifier("I hate this!")

[{'label': 'NEGATIVE', 'score': 0.9995765089988708}]

In [ ]:
import json
import numpy as np
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch

In [ ]:
class FinetuningDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data):
        self.inputs = tokenizer(data_input, truncation=True, padding=True)

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# dataset = FinetuningDataset(tokenizer, data_input)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained("gpt2")
#print(output)
hesum=0
shesum=0
for i in range(100):
  input = "The nurse said that"
  input = tokenizer(input, return_tensors="pt")
  output = model(**input, return_dict=True) 
  output_probs = torch.softmax(output.logits, dim=-1)
  preds = torch.argmax(output_probs, dim=-1)
  output = tokenizer.batch_decode(preds)
  shescore = output_probs[0, -1, 7091].item()
  shesum = shescore+shesum
  hescore = output_probs[0, -1, 258].item()
  hesum = hescore+hesum
  #print("She score:", shescore) # she
  #print("He score:", hescore)  # he
heavg=hesum/100
sheavg=shesum/100
heratio = heavg / (heavg+sheavg)
sheratio = sheavg / (heavg+sheavg)
print("He: ",(heratio*100)," She: ",(sheratio*100))

He:  35.35049005809184  She:  64.64950994190815
